In [1]:
import pandas as pd
import numpy as np
import pymysql
import datetime
import warnings
warnings.filterwarnings('ignore')

## Comportamiento

In [11]:
sql_comportamiento = """
select * 
from
	(select ca.*,cd.saldo_pendiente_total as saldo, 
    cd.total_pagado_capital as 'capital_pagado', 
    cd.saldo_pendiente_capital as 'capital_pendiente',
    cd.dias_atraso,cd.fecha,cd.status,cd.convenio,
	cd.fecha_aut_linea as fecha_inicio_relacion,
    cd.reestructurado,pagos_anticipados_u4p,pagos_en_tiempo_u4p,
    pagos_destiempo_u4p,pagos_pendientes_u4p
	from
	(
	select 
	cas.id_distribuidor, 
	max(cas.id_red) as 'id_red', 
	cas.id_asociado as id_operacion,
	max(cas.tipo_prestamo) as 'tipo_prestamo',
	max(cas.fin_credito) as fecha_cierre,
	max(cas.dispersion) as fecha_de_dispersion
	from (
		select 
		if(tipo_prestamo = 'PRESTAMO PERSONAL RED', id_distribuidor_linea_de_credito, id_distribuidor) as 'id_distribuidor', 
		id_distribuidor as 'id_red',
		tipo_prestamo,
		fin_credito,
		dispersion,
		id_asociado
		from colocacion_asociado
	) as cas 
	group by 
		cas.id_distribuidor, 
		cas.id_asociado
	) ca
	left join cartera_distribuidor cd on ca.id_red = cd.id_distribuidor
	where -- cd.fecha = '2023-08-25'
	fecha_de_dispersion>='2023-03-01'
    and fecha_cierre <='2024-04-15'
	and cd.fecha >= fecha_de_dispersion 
    and cd.fecha<= DATE_ADD(fecha_cierre, INTERVAL 4 MONTH)

) as qry
WHERE
    (DAY(qry.fecha) = 15 or
    qry.fecha = LAST_DAY(qry.fecha))

"""

In [12]:
cnx = pymysql.connect(host='159.89.90.197', port = 3306, user='antonio_diaz', passwd="4nT0ni_0.dZ",
                               charset='utf8',db = 'volana')
cursor = cnx.cursor()
cursor.execute(sql_comportamiento)
data_comportamiento = cursor.fetchall()
data_comportamiento = pd.DataFrame(data_comportamiento, columns=[i[0] for i in cursor.description])
cnx.close()

In [15]:
data_comportamiento.to_csv("./data/data_comportamiento_15_30_dias_nuevo_modelo_2024_04_22.csv", index=False)

In [16]:
data_comportamiento = pd.read_csv("./data/data_comportamiento_15_30_dias_nuevo_modelo_2024_04_22.csv")

In [17]:
data_comportamiento.shape

(152479, 19)

## CC

In [7]:
sql_cc = """
SELECT 	consultas.id_distribuidor, 
        consultas.fecha_consulta,
		cuentas.*
FROM
 (SELECT 
	folioConsulta as 'folio', 
	sum(SaldoVencido) as 'saldo_vencido_total', 
    sum(NumeroPagosVencidos) as 'pagos_vencidos',
    count(folioConsulta) as 'creditos_totales',
    count(ClavePrevencion) as 'creditos_con_claves_prevencion',
    COUNT(IF(SaldoVencido = 0, NULL, 1)) as 'creditos_completados_con_atraso',
    COUNT(IF(SaldoVencido = 0, 1, NULL)) as 'creditos_completados_sin_atraso'
 FROM cc_prospectos_creditos 
 group by folioConsulta ) as cuentas 
 left join (select 
			 folioConsulta as 'folio', max(idCliente) as 'id_distribuidor',
             max(dateTime) as 'fecha_consulta'
			 
			 from cc_prospectos_folios group by folioConsulta) as consultas
 ON cuentas.folio = consultas.folio
"""

In [8]:
cnx = pymysql.connect(host='159.89.90.197', port = 3306, user='antonio_diaz', passwd="4nT0ni_0.dZ",
                               charset='utf8',db = 'volana')
cursor = cnx.cursor()
cursor.execute(sql_cc)
data_cc = cursor.fetchall()
data_cc = pd.DataFrame(data_cc, columns=[i[0] for i in cursor.description])
cnx.close()

In [18]:
data_cc.to_csv("./data/data_cc_nuevo_modelo_2024_04_22.csv", index=False)

In [19]:
data_cc = pd.read_csv("./data/data_cc_nuevo_modelo_2024_04_22.csv")

## Validando que todo exista para reportarse

In [13]:
"""
Revisar id por id_distribuidor
Después crédito por crédito
Por cada crédito revisar qué si hay algun folio de ese mismo id_distribuidor que esté
máximo 15 días atrás y marcarlo, si hay, ir concatando a un DF nuevo
Después, eliminar de data_cc los folios que no se relacionaron a nada
""";

In [20]:
data_comportamiento.fecha_de_dispersion = data_comportamiento.fecha_de_dispersion.astype(str)

In [21]:
data_comportamiento = data_comportamiento.sort_values(by = 'id_distribuidor', ascending=False)

In [22]:
from tqdm import tqdm

In [23]:
data_comportamiento_enviar = pd.DataFrame([])
total = 0

for id_distribuidor in tqdm(data_comportamiento.id_distribuidor.unique()):
    temp = data_comportamiento.query(f"id_distribuidor == {id_distribuidor}")
    
    for fecha_dispersion_credito in temp.fecha_de_dispersion.unique():
        total += 1
        fecha_dispersion_credito_date = datetime.datetime.strptime(fecha_dispersion_credito, '%Y-%m-%d').date()
        fecha_dispersion_credito_15_dias_antes = fecha_dispersion_credito_date - datetime.timedelta(days = 45)
        fecha_dispersion_credito_15_dias_antes = fecha_dispersion_credito_15_dias_antes.strftime('%Y-%m-%d')
        
        data_cc_temp = data_cc.query(f"id_distribuidor == {id_distribuidor}") 
        data_cc_temp = data_cc_temp.query(f"fecha_consulta >= '{fecha_dispersion_credito_15_dias_antes}' and fecha_consulta <= '{fecha_dispersion_credito}'")
        
        if data_cc_temp.shape[0] != 0:
            total += 1
            if total % 500 == 0:
                print(f"{id_distribuidor} - {data_cc_temp.folio.iloc[0]}")
            # entonces hay al menos una consulta
            df_resultado_agregar = temp.query(f"fecha_de_dispersion == '{fecha_dispersion_credito}'")
            df_resultado_agregar['folio'] = data_cc_temp.folio.iloc[0]
            
            data_comportamiento_enviar = pd.concat([data_comportamiento_enviar, df_resultado_agregar], axis = 0)
            

 21%|████████████████▎                                                             | 1003/4782 [00:11<00:43, 86.41it/s]

157363 - 1186723003


100%|██████████████████████████████████████████████████████████████████████████████| 4782/4782 [01:15<00:00, 63.29it/s]


In [24]:
data_comportamiento_enviar.tipo_prestamo.value_counts()

PRESTAMO PERSONAL RED     23149
LINEA                      4109
PRESTAMO PERSONAL CLUB      183
Name: tipo_prestamo, dtype: int64

In [25]:
data_comportamiento_enviar.to_csv("./data/enviar/data_comportamiento_15_30_dias_nuevo_modelo_2024_04_22.csv", index=False)

In [26]:
# Quitando los folios que no hacen sentido
data_cc_enviar = data_cc.query("folio in @data_comportamiento_enviar.folio.unique()")

data_cc_enviar.to_csv("./data/enviar/data_cc_nuevo_modelo_2024_04_22.csv")

In [28]:
print(np.setdiff1d(data_cc_enviar.folio, data_comportamiento_enviar.folio))
np.setdiff1d(data_comportamiento_enviar.folio, data_cc_enviar.folio)

[]


array([], dtype=int64)

In [29]:
#data_comportamiento_enviar.query("tipo_prestamo == 'PRESTAMO PERSONAL RED' and id_distribuidor == 11429").sort_values(by = 'fecha')
data_comportamiento_enviar.query("tipo_prestamo == 'PRESTAMO PERSONAL RED'").groupby("id_distribuidor").fecha_de_dispersion.nunique().sort_values()

id_distribuidor
11429     1
159058    1
159027    1
159026    1
159025    1
         ..
143838    2
114568    2
114566    2
154897    2
155889    2
Name: fecha_de_dispersion, Length: 1997, dtype: int64

In [30]:
data_comportamiento_enviar.query("tipo_prestamo == 'PRESTAMO PERSONAL RED'").id_distribuidor.nunique()

1997

In [31]:
data_cc_enviar

,id_distribuidor,fecha_consulta,folio,saldo_vencido_total,pagos_vencidos,creditos_totales,creditos_con_claves_prevencion,creditos_completados_con_atraso,creditos_completados_sin_atraso
0,154610.0,2023-06-07 17:25:26,1166269370,37981,72,10,5,3,7
4,154767.0,2023-06-07 21:38:05,1166502669,9573,36,2,2,2,0
6,154772.0,2023-06-07 22:41:56,1166549975,2253,73,37,23,5,32
7,154771.0,2023-06-07 22:52:35,1166557426,0,5,39,27,0,39
8,122546.0,2023-06-07 23:08:09,1166568805,51870,33,29,22,3,26
...,...,...,...,...,...,...,...,...,...
6921,159050.0,2023-11-17 20:13:13,1281357023,1812,3,21,16,1,20
6938,159760.0,2023-11-17 20:14:03,1281357796,12315,37,11,6,3,8
6951,159769.0,2023-11-17 20:14:32,1281358206,48807,333,292,45,7,285
7417,5812.0,2023-11-21 20:38:40,1284576731,50794,105,12,7,5,7


In [159]:
cnx = pymysql.connect(host='159.89.90.197', port = 3306, user='antonio_diaz', passwd="4nT0ni_0.dZ",
                                   charset='utf8',db = 'volana')
for table in ["creditos", "demograficos", "domicilios", "folios", "mensajes", "persona", "scores"]:
    print(f"sigue {table}")
    sql = f"select * from cc_prospectos_{table}"
    
    cursor = cnx.cursor()
    cursor.execute(sql)
    data_cc_prospectos = cursor.fetchall()
    data_cc_prospectos = pd.DataFrame(data_cc_prospectos, columns=[i[0] for i in cursor.description])
    data_cc_prospectos.to_csv(f"./data/enviar/cc_prospectos_{table}.csv", index = False)
    print(f"listin {table}\n")
cnx.close()

sigue creditos
listin creditos

sigue demograficos
listin demograficos

sigue domicilios
listin domicilios

sigue folios
listin folios

sigue mensajes
listin mensajes

sigue persona
listin persona

sigue scores
listin scores



## Revisando el tema

In [32]:
import pandas as pd
data = pd.read_csv("./data/enviar/data_comportamiento_15_30_dias_nuevo_modelo_2024_04_22.csv")
data_cc = pd.read_csv("./data/enviar/data_cc_nuevo_modelo_2024_04_22.csv")

In [33]:
temp = data[['fecha_de_dispersion', 'folio']].merge(data_cc[['fecha_consulta', 'folio']], on = 'folio')
temp.query("fecha_de_dispersion < fecha_consulta")

,fecha_de_dispersion,folio,fecha_consulta


In [34]:
(pd.to_datetime(temp.fecha_consulta) > pd.to_datetime(temp.fecha_de_dispersion)).sum()

0

In [35]:
data.shape

(27441, 20)

In [36]:
data_cc

,Unnamed: 0,id_distribuidor,fecha_consulta,folio,saldo_vencido_total,pagos_vencidos,creditos_totales,creditos_con_claves_prevencion,creditos_completados_con_atraso,creditos_completados_sin_atraso
0,0,154610.0,2023-06-07 17:25:26,1166269370,37981,72,10,5,3,7
1,4,154767.0,2023-06-07 21:38:05,1166502669,9573,36,2,2,2,0
2,6,154772.0,2023-06-07 22:41:56,1166549975,2253,73,37,23,5,32
3,7,154771.0,2023-06-07 22:52:35,1166557426,0,5,39,27,0,39
4,8,122546.0,2023-06-07 23:08:09,1166568805,51870,33,29,22,3,26
...,...,...,...,...,...,...,...,...,...,...
2107,6921,159050.0,2023-11-17 20:13:13,1281357023,1812,3,21,16,1,20
2108,6938,159760.0,2023-11-17 20:14:03,1281357796,12315,37,11,6,3,8
2109,6951,159769.0,2023-11-17 20:14:32,1281358206,48807,333,292,45,7,285
2110,7417,5812.0,2023-11-21 20:38:40,1284576731,50794,105,12,7,5,7


In [23]:
data_cc_creditos = pd.read_csv("./data/enviar/cc_prospectos_creditos.csv")

C:\Users\CIENTIFICO DE DATOS\AppData\Local\Temp\ipykernel_5360\3809239176.py:1: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  data_cc_creditos = pd.read_csv("./data/enviar/cc_prospectos_creditos.csv")


In [28]:
data_cc_creditos.columns

Index(['fechaActualizacion', 'registroImpugnado', 'nombreOtorgante',
       'tipoResponsabilidad', 'tipoCuenta', 'tipoCredito',
       'claveUnidadMonetaria', 'valorActivoValuacion', 'numeroPagos',
       'frecuenciaPagos', 'montoPagar', 'fechaAperturaCuenta',
       'fechaUltimoPago', 'fechaUltimaCompra', 'fechaCierreCuenta',
       'fechaReporte', 'creditoMaximo', 'saldoActual', 'limiteCredito',
       'saldoVencido', 'numeroPagosVencidos', 'pagoActual', 'historicoPagos',
       'totalPagosReportados', 'peorAtraso', 'saldoVencidoPeorAtraso',
       'clavePrevencion', 'fechaPeorAtraso', 'ultimaFechaSaldoCero',
       'folioConsulta', 'cuentaActual'],
      dtype='object')

In [30]:
data_cc_creditos.merge(data, right_on="folio", left_on='folioConsulta').columns

Index(['fechaActualizacion', 'registroImpugnado', 'nombreOtorgante',
       'tipoResponsabilidad', 'tipoCuenta', 'tipoCredito',
       'claveUnidadMonetaria', 'valorActivoValuacion', 'numeroPagos',
       'frecuenciaPagos', 'montoPagar', 'fechaAperturaCuenta',
       'fechaUltimoPago', 'fechaUltimaCompra', 'fechaCierreCuenta',
       'fechaReporte', 'creditoMaximo', 'saldoActual', 'limiteCredito',
       'saldoVencido', 'numeroPagosVencidos', 'pagoActual', 'historicoPagos',
       'totalPagosReportados', 'peorAtraso', 'saldoVencidoPeorAtraso',
       'clavePrevencion', 'fechaPeorAtraso', 'ultimaFechaSaldoCero',
       'folioConsulta', 'cuentaActual', 'id_distribuidor', 'id_red',
       'id_operacion', 'tipo_prestamo', 'fecha_cierre', 'fecha_de_dispersion',
       'saldo', 'capital_pagado', 'capital_pendiente', 'dias_atraso', 'fecha',
       'status', 'convenio', 'fecha_inicio_relacion', 'reestructurado',
       'pagos_anticipados_u4p', 'pagos_en_tiempo_u4p', 'pagos_destiempo_u4p',
    

In [42]:
#data.reestructurado.value_counts()
data.query("reestructurado == 'NO' and tipo_prestamo == 'PRESTAMO PERSONAL CLUB'").sort_values(by = ['id_distribuidor','fecha'])

,id_distribuidor,id_red,id_operacion,tipo_prestamo,fecha_cierre,fecha_de_dispersion,saldo,capital_pagado,capital_pendiente,dias_atraso,fecha,status,convenio,fecha_inicio_relacion,reestructurado,pagos_anticipados_u4p,pagos_en_tiempo_u4p,pagos_destiempo_u4p,pagos_pendientes_u4p,folio
27393,116,116,155523,PRESTAMO PERSONAL CLUB,2023-12-15,2023-06-19,453846,5510099.56,338243.62,0,2023-06-30,DISTRIBUIDOR,NO,2017-05-29,NO,4.0,0.0,0.0,0.0,1172294613
27392,116,116,155523,PRESTAMO PERSONAL CLUB,2023-12-15,2023-06-19,494005,5576955.38,364888.30,0,2023-07-15,DISTRIBUIDOR,NO,2017-05-29,NO,4.0,0.0,0.0,0.0,1172294613
27391,116,116,155523,PRESTAMO PERSONAL CLUB,2023-12-15,2023-06-19,532914,5643374.55,394969.33,0,2023-07-31,DISTRIBUIDOR,NO,2017-05-29,NO,4.0,0.0,0.0,0.0,1172294613
27394,116,116,155523,PRESTAMO PERSONAL CLUB,2023-12-15,2023-06-19,512713,5713633.47,384710.57,0,2023-08-15,DISTRIBUIDOR,NO,2017-05-29,NO,3.0,1.0,0.0,0.0,1172294613
27395,116,116,155523,PRESTAMO PERSONAL CLUB,2023-12-15,2023-06-19,531307,5789677.81,396166.76,0,2023-08-31,DISTRIBUIDOR,NO,2017-05-29,NO,3.0,1.0,0.0,0.0,1172294613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17962,144578,144578,156507,PRESTAMO PERSONAL CLUB,2023-12-31,2023-06-28,87616,129399.59,65101.75,0,2024-01-15,DISTRIBUIDOR,NO,2023-03-13,NO,4.0,0.0,0.0,0.0,1180565161
17954,144578,144578,156507,PRESTAMO PERSONAL CLUB,2023-12-31,2023-06-28,86067,140367.71,64633.68,0,2024-01-31,DISTRIBUIDOR,NO,2023-03-13,NO,4.0,0.0,0.0,0.0,1180565161
17948,144578,144578,156507,PRESTAMO PERSONAL CLUB,2023-12-31,2023-06-28,75093,152100.99,57400.44,0,2024-02-15,DISTRIBUIDOR,NO,2023-03-13,NO,3.0,1.0,0.0,0.0,1180565161
17946,144578,144578,156507,PRESTAMO PERSONAL CLUB,2023-12-31,2023-06-28,85038,168819.08,63182.42,0,2024-02-29,DISTRIBUIDOR,NO,2023-03-13,NO,3.0,1.0,0.0,0.0,1180565161


In [38]:
data.tipo_prestamo.value_counts()

PRESTAMO PERSONAL RED     23149
LINEA                      4109
PRESTAMO PERSONAL CLUB      183
Name: tipo_prestamo, dtype: int64

## Cruzando con la info de cartera de JC

In [47]:
data_comportamiento_enviar = pd.read_csv("./data/enviar/data_comportamiento_15_30_dias_nuevo_modelo_2024_04_22.csv")
data_comportamiento_enviar.head(1)

,id_distribuidor,id_red,id_operacion,tipo_prestamo,fecha_cierre,fecha_de_dispersion,saldo,capital_pagado,capital_pendiente,dias_atraso,fecha,status,convenio,fecha_inicio_relacion,reestructurado,pagos_anticipados_u4p,pagos_en_tiempo_u4p,pagos_destiempo_u4p,pagos_pendientes_u4p,folio
0,169239,169239,172348,LINEA,2024-01-31,2023-12-13,25702,13229.8,18770.16,0,2024-03-15,DISTRIBUIDOR,NO,2023-11-28,NO,0.0,2.0,1.0,1.0,1278752802


In [68]:
data_comportamiento_enviar.query("id_distribuidor != id_red").id_distribuidor.unique()

array([167423, 167045, 166922, ...,  84751,  65981,  11429], dtype=int64)

In [69]:
data_comportamiento_enviar.query("id_distribuidor == 166922")

,id_distribuidor,id_red,id_operacion,tipo_prestamo,fecha_cierre,fecha_de_dispersion,saldo,capital_pagado,capital_pendiente,dias_atraso,fecha,status,convenio,fecha_inicio_relacion,reestructurado,pagos_anticipados_u4p,pagos_en_tiempo_u4p,pagos_destiempo_u4p,pagos_pendientes_u4p,folio
19,166922,171125,171127,PRESTAMO PERSONAL RED,2024-01-11,2023-11-30,47910,0.0,42212.2,39,2024-01-15,DISTRIBUIDOR RED,NO,2023-11-30,NO,NaN,NaN,NaN,NaN,1259743646
20,166922,171125,171127,PRESTAMO PERSONAL RED,2024-01-11,2023-11-30,47910,0.0,42212.2,0,2023-11-30,DISTRIBUIDOR RED,NO,2023-11-30,NO,0.0,0.0,0.0,0.0,1259743646
21,166922,171125,171127,PRESTAMO PERSONAL RED,2024-01-11,2023-11-30,47910,0.0,42212.2,8,2023-12-15,DISTRIBUIDOR RED,NO,2023-11-30,NO,NaN,NaN,NaN,NaN,1259743646
22,166922,171125,171127,PRESTAMO PERSONAL RED,2024-01-11,2023-11-30,47910,0.0,42212.2,24,2023-12-31,DISTRIBUIDOR RED,NO,2023-11-30,NO,0.0,0.0,0.0,4.0,1259743646
23,166922,171125,171127,PRESTAMO PERSONAL RED,2024-01-11,2023-11-30,47910,0.0,42212.2,55,2024-01-31,DISTRIBUIDOR RED,NO,2023-11-30,NO,NaN,NaN,NaN,NaN,1259743646
24,166922,171125,171127,PRESTAMO PERSONAL RED,2024-01-11,2023-11-30,47910,0.0,42212.2,99,2024-03-15,DISTRIBUIDOR RED,NO,2023-11-30,NO,NaN,NaN,NaN,NaN,1259743646


In [44]:
data_cartera_perdida_cosecha = pd.read_excel("./data/240411 PERDIDA DE COSECHA_.xlsb", 
                                             sheet_name = 'COLOCACION', header=1)

In [48]:
data_cartera_perdida_cosecha.columns

Index(['COMPAÑIA', 'SUCURSAL', 'ID ASESOR', 'ASESOR', 'ID DIST.',
       'DISTRIBUIDOR', 'ID ASOCIADO', 'ASOCIADO', 'NO.PAGOS', 'CICLO CREDITO',
       'PAGO QUINCENAL', 'INICIO CREDITO', 'FIN CREDITO', 'DISPERSION',
       'CAPITAL', 'INTERES', 'IVA', 'SEGURO', 'EMPLEADO ASESOR', 'TASA',
       'ESTATUS', 'TIPO PRODUCTO', 'ID DISTRIBUIDOR LINEA DE CREDITO',
       'TIPO DISPERSION', 'GARANTIA', 'ID CLIENTE', 'ORIGEN', 'ID PREVIO',
       'TIPO DISTRIBUIDOR', 'CAMBIO DE ESTATUS FINAL', 'DESTINO CREDITO',
       'REESTRUCTURA', 'SEMANA 12 DE LA RED', 'FECHA FIN DE CREDITO',
       'CERRADO', 'MES DE COLOCACIÓN', 'MES DE CIERRE DEL CRÉDITO',
       'CICLO DEL CRÉDITO', 'PRODUCTO', 'CONSECUTIVO', 'año', 'mes', 'año|mes',
       'REGION', 'ID CLIENTE.1', 'ID CLIENTE | CICLO PARA RENOVAR',
       'ID CLIENTE | CICLO RENOVADO', 'RENOVACION', 'ESTATUS.', 'DIAS ATRASO',
       'F. ULTIMO PAGO', 'CAPITAL PAGADO', 'CAPITAL PENDIENTE', 'PAGO FINAL',
       'PERDIDA DE COSECHA', 'REGION.1', 'ZONA'

In [52]:
# Filtro solo los de prestamo red
data_cartera_perdida_cosecha = data_cartera_perdida_cosecha[data_cartera_perdida_cosecha['TIPO PRODUCTO'] == "PRESTAMO PERSONAL RED"]

In [57]:
# Solo ciclo 1
data_cartera_perdida_cosecha = data_cartera_perdida_cosecha[data_cartera_perdida_cosecha['CICLO DEL CRÉDITO'] == 1]

In [75]:
from datetime import datetime, timedelta

def excel_to_date(excel_date_value):
    # Fecha base en Excel (fecha de inicio de Excel: 1 de enero de 1900)
    excel_base_date = datetime(1899, 12, 30)

    # Calcula la diferencia en días desde la fecha base de Excel
    delta = timedelta(days=excel_date_value)

    # Calcula la fecha final
    fecha = excel_base_date + delta

    return fecha

data_cartera_perdida_cosecha['FIN CREDITO'] = data_cartera_perdida_cosecha['FIN CREDITO'].apply(excel_to_date)

In [76]:
data_cartera_perdida_cosecha = data_cartera_perdida_cosecha[['ID DIST.',
                              'PERDIDA DE COSECHA','CAPITAL',
                              'CAPITAL PAGADO', 'CAPITAL PENDIENTE', 
                              'DIAS ATRASO', 'FIN CREDITO',
                             'ID DISTRIBUIDOR LINEA DE CREDITO']]

data_cartera_perdida_cosecha.rename(columns={
"ID DISTRIBUIDOR LINEA DE CREDITO":"id_distribuidor",
'ID DIST.': "id_red"
}, inplace=True)

In [79]:
data_cartera_perdida_cosecha.columns = [col.lower().replace(" ","_") for col in data_cartera_perdida_cosecha.columns]

In [83]:
data_cartera_perdida_cosecha.id_distribuidor = data_cartera_perdida_cosecha.id_distribuidor.astype(int)

In [84]:
data_cartera_perdida_cosecha

,id_red,perdida_de_cosecha,capital,capital_pagado,capital_pendiente,dias_atraso,fin_credito,id_distribuidor
33,139409,-0.206619,6000.00,33322.00,8678.00,329,2023-06-15,139215
34,139409,-0.206619,6000.00,33322.00,8678.00,329,2023-06-15,139211
35,139409,-0.206619,6000.00,33322.00,8678.00,329,2023-06-15,139351
126,139628,0.000000,6000.00,46000.00,0.00,0,2023-04-27,138931
196,139004,0.000000,8000.00,94000.01,0.00,0,2023-04-28,138983
...,...,...,...,...,...,...,...,...
24900,182478,0.000000,5000.04,0.00,51999.99,0,2024-07-31,182487
24901,182478,0.000000,5000.04,0.00,51999.99,0,2024-07-31,182488
24910,183599,0.000000,9999.99,0.00,77999.76,0,2024-07-31,182999
24936,183038,0.000000,7999.99,0.00,54000.04,0,2024-07-31,183289


In [102]:
data_comportamiento_enviar

,id_distribuidor,id_red,id_operacion,tipo_prestamo,fecha_cierre,fecha_de_dispersion,saldo,capital_pagado,capital_pendiente,dias_atraso,fecha,status,convenio,fecha_inicio_relacion,reestructurado,pagos_anticipados_u4p,pagos_en_tiempo_u4p,pagos_destiempo_u4p,pagos_pendientes_u4p,folio
0,169239,169239,172348,LINEA,2024-01-31,2023-12-13,25702,13229.80,18770.16,0,2024-03-15,DISTRIBUIDOR,NO,2023-11-28,NO,0.0,2.0,1.0,1.0,1278752802
1,169239,169239,172348,LINEA,2024-01-31,2023-12-13,20735,10067.11,14932.89,0,2024-02-29,DISTRIBUIDOR,NO,2023-11-28,NO,0.0,3.0,0.0,1.0,1278752802
2,169239,169239,172348,LINEA,2024-01-31,2023-12-13,20935,7383.28,14616.71,0,2024-02-15,DISTRIBUIDOR,NO,2023-11-28,NO,1.0,2.0,0.0,1.0,1278752802
3,169239,169239,172348,LINEA,2024-01-31,2023-12-13,21145,4194.99,14804.96,0,2024-01-31,DISTRIBUIDOR,NO,2023-11-28,NO,1.0,1.0,0.0,1.0,1278752802
4,169239,169239,172348,LINEA,2024-01-31,2023-12-13,19005,1632.88,13367.04,0,2023-12-31,DISTRIBUIDOR,NO,2023-11-28,NO,1.0,0.0,0.0,0.0,1278752802
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27436,116,116,155713,LINEA,2023-10-15,2023-06-20,470085,5939730.66,354114.42,0,2023-09-30,DISTRIBUIDOR,NO,2017-05-29,NO,2.0,2.0,0.0,0.0,1172294613
27437,116,116,155713,LINEA,2023-10-15,2023-06-20,585247,5789677.81,432166.90,0,2023-09-15,DISTRIBUIDOR,NO,2017-05-29,NO,2.0,1.0,0.0,1.0,1172294613
27438,116,116,155713,LINEA,2023-10-15,2023-06-20,531307,5789677.81,396166.76,0,2023-08-31,DISTRIBUIDOR,NO,2017-05-29,NO,3.0,1.0,0.0,0.0,1172294613
27439,116,116,155713,LINEA,2023-10-15,2023-06-20,512713,5713633.47,384710.57,0,2023-08-15,DISTRIBUIDOR,NO,2017-05-29,NO,3.0,1.0,0.0,0.0,1172294613


In [101]:
c = 0
for i, row in data_comportamiento_enviar.iterrows():
    #print(row['id_distribuidor'])
    if row['fecha_cierre'] == data_cartera_perdida_cosecha.query(f"id_distribuidor == {row['id_distribuidor']}").fin_credito.astype(str).iloc[0]:
        print("p ",row['fecha_cierre'])
        print("ene ",data_cartera_perdida_cosecha.query(f"id_distribuidor == {row['id_distribuidor']}").fin_credito.astype(str).iloc[0])
        c += 1
    
    if c > 100:
        break
    #break

p  2024-02-01
ene  2024-02-01
p  2024-02-01
ene  2024-02-01
p  2024-02-01
ene  2024-02-01
p  2024-02-01
ene  2024-02-01
p  2024-02-01
ene  2024-02-01
p  2024-02-01
ene  2024-02-01
p  2024-02-01
ene  2024-02-01
p  2024-02-01
ene  2024-02-01
p  2024-02-01
ene  2024-02-01
p  2024-01-29
ene  2024-01-29
p  2024-01-29
ene  2024-01-29
p  2024-01-29
ene  2024-01-29
p  2024-01-29
ene  2024-01-29
p  2024-01-29
ene  2024-01-29
p  2024-01-29
ene  2024-01-29
p  2024-01-29
ene  2024-01-29
p  2024-01-29
ene  2024-01-29
p  2024-01-29
ene  2024-01-29
p  2024-01-31
ene  2024-01-31
p  2024-01-31
ene  2024-01-31
p  2024-01-31
ene  2024-01-31
p  2024-01-31
ene  2024-01-31
p  2024-01-31
ene  2024-01-31
p  2024-01-31
ene  2024-01-31
p  2024-01-31
ene  2024-01-31
p  2024-01-31
ene  2024-01-31
p  2024-01-31
ene  2024-01-31
p  2024-01-31
ene  2024-01-31
p  2024-01-31
ene  2024-01-31
p  2024-01-31
ene  2024-01-31
p  2024-01-31
ene  2024-01-31
p  2024-01-31
ene  2024-01-31
p  2024-01-31
ene  2024-01-31
p  2024-01

In [99]:
c

101

In [90]:
row['fecha_cierre'] == data_cartera_perdida_cosecha.query("id_distribuidor == 169239").fin_credito.astype(str).iloc[0]

'2024-01-31'

In [94]:
data_cartera_perdida_cosecha.query("id_distribuidor == 169239").fin_credito.astype(str).iloc[0]

'2024-03-20'

In [105]:
data_comportamiento_enviar.tipo_prestamo.value_counts()

PRESTAMO PERSONAL RED     23149
LINEA                      4109
PRESTAMO PERSONAL CLUB      183
Name: tipo_prestamo, dtype: int64

In [115]:
data_comportamiento_enviar.query("tipo_prestamo == 'PRESTAMO PERSONAL RED'").groupby(['id_distribuidor']).first()

,id_red,id_operacion,tipo_prestamo,fecha_cierre,fecha_de_dispersion,saldo,capital_pagado,capital_pendiente,dias_atraso,fecha,status,convenio,fecha_inicio_relacion,reestructurado,pagos_anticipados_u4p,pagos_en_tiempo_u4p,pagos_destiempo_u4p,pagos_pendientes_u4p,folio
id_distribuidor,,,,,,,,,,,,,,,,,,,
11429,162969,162972,PRESTAMO PERSONAL RED,2023-12-20,2023-08-30,37730,14320.33,30679.60,0,2023-10-15,DISTRIBUIDOR RED,NO,2023-08-30,NO,0.0,0.0,0.0,0.0,1221093519
65981,163078,163292,PRESTAMO PERSONAL RED,2023-12-21,2023-08-31,57232,0.00,44000.14,0,2023-08-31,DISTRIBUIDOR RED,NO,2023-08-31,NO,0.0,0.0,0.0,0.0,1225640832
84751,165675,165676,PRESTAMO PERSONAL RED,2024-01-19,2023-09-29,16348,60170.30,14829.69,26,2024-03-15,DISTRIBUIDOR RED,NO,2023-09-29,SI,0.0,0.0,4.0,0.0,1245799171
86322,164549,164553,PRESTAMO PERSONAL RED,2024-01-09,2023-09-19,82920,3703.90,67296.14,0,2023-09-30,DISTRIBUIDOR RED,NO,2023-09-19,NO,1.0,0.0,0.0,0.0,1237020495
86345,164549,164552,PRESTAMO PERSONAL RED,2024-01-09,2023-09-19,0,71000.04,0.00,0,2024-01-15,CERRADO,NO,2023-09-19,NO,0.0,1.0,3.0,0.0,1237022303
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166390,166475,166481,PRESTAMO PERSONAL RED,2024-02-01,2023-10-12,58869,30431.44,49568.43,0,2023-11-30,DISTRIBUIDOR RED,NO,2023-10-12,NO,4.0,0.0,0.0,0.0,1255358308
166913,171125,171126,PRESTAMO PERSONAL RED,2024-01-11,2023-11-30,47910,0.00,42212.20,99,2024-03-15,DISTRIBUIDOR RED,NO,2023-11-30,NO,0.0,0.0,0.0,4.0,1259742923
166922,171125,171127,PRESTAMO PERSONAL RED,2024-01-11,2023-11-30,47910,0.00,42212.20,39,2024-01-15,DISTRIBUIDOR RED,NO,2023-11-30,NO,0.0,0.0,0.0,0.0,1259743646


In [111]:
data_comportamiento_enviar.sort_values("fecha_cierre").drop_duplicates(['id_distribuidor'], keep='first')

,id_distribuidor,id_red,id_operacion,tipo_prestamo,fecha_cierre,fecha_de_dispersion,saldo,capital_pagado,capital_pendiente,dias_atraso,fecha,status,convenio,fecha_inicio_relacion,reestructurado,pagos_anticipados_u4p,pagos_en_tiempo_u4p,pagos_destiempo_u4p,pagos_pendientes_u4p,folio
26160,59982,59982,156777,LINEA,2023-08-31,2023-06-30,86344,308601.07,62219.40,0,2023-07-15,DISTRIBUIDOR,NO,2020-10-09,NO,0.0,3.0,0.0,1.0,1174969554
20254,135039,135039,157319,LINEA,2023-08-31,2023-07-05,18945,28439.25,14560.69,0,2023-09-30,DISTRIBUIDOR,NO,2022-11-23,NO,4.0,0.0,0.0,0.0,1175193026
16363,147733,147733,159002,LINEA,2023-08-31,2023-07-21,4875,5137.76,3862.25,0,2023-07-31,DISTRIBUIDOR,NO,2023-03-31,NO,2.0,0.0,1.0,1.0,1192693788
24807,98597,98597,159213,LINEA,2023-09-15,2023-07-25,0,82000.05,0.00,0,2023-12-31,DISTRIBUIDOR,NO,2021-12-03,NO,3.0,1.0,0.0,0.0,1177115923
16292,147757,147757,160092,LINEA,2023-09-15,2023-07-31,12455,26531.36,9468.70,12,2023-10-15,DISTRIBUIDOR,NO,2023-03-31,NO,0.0,1.0,1.0,2.0,1192693536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,165029,166422,166445,PRESTAMO PERSONAL RED,2024-01-31,2023-10-11,0,48999.77,0.00,0,2024-01-31,CERRADO,NO,2023-10-11,NO,NaN,NaN,NaN,NaN,1241777566
2427,162731,164809,164811,PRESTAMO PERSONAL RED,2024-01-31,2023-10-11,47520,0.00,34999.60,0,2023-10-15,DISTRIBUIDOR RED,NO,2023-10-11,NO,NaN,NaN,NaN,NaN,1224914605
2133,163250,164809,164812,PRESTAMO PERSONAL RED,2024-01-31,2023-10-11,0,34999.60,0.00,0,2024-01-31,CERRADO,NO,2023-10-11,NO,NaN,NaN,NaN,NaN,1227023491
31,166390,166475,166481,PRESTAMO PERSONAL RED,2024-02-01,2023-10-12,58869,30431.44,49568.43,0,2023-11-30,DISTRIBUIDOR RED,NO,2023-10-12,NO,4.0,0.0,0.0,0.0,1255358308


In [119]:
data_comportamiento_enviar = data_comportamiento_enviar.sort_values("fecha_de_dispersion")

In [134]:
data_comportamiento_enviar['ciclo'] = 1

In [151]:
for id_ in data_comportamiento_enviar.id_distribuidor.unique():
    cicl_ = 1
    for fec in data_comportamiento_enviar.query(f"id_distribuidor == {id_}").fecha_cierre.unique():
        data_comportamiento_enviar.loc[(data_comportamiento_enviar.id_distribuidor == id_) & (data_comportamiento_enviar.fecha_cierre == fec), 'ciclo']= cicl_
        cicl_+=1

In [154]:
data_comportamiento_enviar

,id_distribuidor,id_red,id_operacion,tipo_prestamo,fecha_cierre,fecha_de_dispersion,saldo,capital_pagado,capital_pendiente,dias_atraso,...,status,convenio,fecha_inicio_relacion,reestructurado,pagos_anticipados_u4p,pagos_en_tiempo_u4p,pagos_destiempo_u4p,pagos_pendientes_u4p,folio,ciclo
12813,154771,154588,154839,PRESTAMO PERSONAL RED,2023-09-28,2023-06-08,121701,21066.10,103933.77,0,...,DISTRIBUIDOR RED,NO,2023-06-08,SI,1.0,0.0,2.0,0.0,1166557426,1
22952,122546,154588,154832,PRESTAMO PERSONAL RED,2023-09-28,2023-06-08,0,124999.87,0.00,0,...,DISTRIBUIDOR RED,NO,2023-06-08,SI,NaN,NaN,NaN,NaN,1166568805,1
22951,122546,154588,154832,PRESTAMO PERSONAL RED,2023-09-28,2023-06-08,0,124999.87,0.00,0,...,DISTRIBUIDOR RED,NO,2023-06-08,SI,4.0,0.0,0.0,0.0,1166568805,1
22950,122546,154588,154832,PRESTAMO PERSONAL RED,2023-09-28,2023-06-08,0,124999.87,0.00,0,...,DISTRIBUIDOR RED,NO,2023-06-08,SI,NaN,NaN,NaN,NaN,1166568805,1
22949,122546,154588,154832,PRESTAMO PERSONAL RED,2023-09-28,2023-06-08,0,124999.87,0.00,0,...,DISTRIBUIDOR RED,NO,2023-06-08,SI,NaN,NaN,NaN,NaN,1166568805,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,169239,169239,172348,LINEA,2024-01-31,2023-12-13,19005,1632.88,13367.04,0,...,DISTRIBUIDOR,NO,2023-11-28,NO,1.0,0.0,0.0,0.0,1278752802,1
3,169239,169239,172348,LINEA,2024-01-31,2023-12-13,21145,4194.99,14804.96,0,...,DISTRIBUIDOR,NO,2023-11-28,NO,1.0,1.0,0.0,1.0,1278752802,1
2,169239,169239,172348,LINEA,2024-01-31,2023-12-13,20935,7383.28,14616.71,0,...,DISTRIBUIDOR,NO,2023-11-28,NO,1.0,2.0,0.0,1.0,1278752802,1
1,169239,169239,172348,LINEA,2024-01-31,2023-12-13,20735,10067.11,14932.89,0,...,DISTRIBUIDOR,NO,2023-11-28,NO,0.0,3.0,0.0,1.0,1278752802,1


In [155]:
data_comportamiento_enviar.to_csv("./data/enviar/data_comportamiento_15_30_dias_nuevo_modelo_2024_04_22_ciclos.csv")